In [13]:
import re # smi 파일에서 텍스트 뽑아내는 모듈
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize


def smi_to_df (file_path):
    file = open(file_path, 'r', encoding = 'cp949')
    wholefile = file.readlines()
    # 모든 대문자를 소문자로 변환
    wholefile_lower = wholefile.lower()
    # 타임스템프와 본문 분리
    pattern = re.compile(r'<sync start=(\d+)><p class=krcc>\n(.*?)\n', re.DOTALL)

    matches = pattern.findall(''.join(wholefile_lower))

    df = pd.DataFrame(matches, columns=['Time_Stamp(s)', 'Text'])
    df['Time_Stamp(s)'] = df['Time_Stamp(s)'].apply(lambda x: float(x)/1000)

    # Text를 하나의 문장으로 합치기
    text = ''.join(df['Text'].tolist())
    sentences = sent_tokenize(''.join(text))
    nltk_df = pd.DataFrame(sentences, columns=['Text'])

    return df, nltk_df


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
from konlpy.tag import Komoran
from collections import Counter
import pandas as pd

import plotly.graph_objects as go
import os

## 용어사전 정의
# 불용어
with open('./stopword.txt', 'r', encoding='utf-8') as stopwords_file:
    # 파일 내용을 줄 단위로 읽어와 리스트로 저장
    stopwords = stopwords_file.read().splitlines()

## 용어사전
# 두 파일의 내용을 읽어와서 합치기 (수업과 관련된 단어, 관련없는 단어) ## 애매한 단어 추가 예정
with open('./user_dic.txt', 'r', encoding='utf-8') as file1, open('./user_dic_moo.txt', 'r', encoding='utf-8') as file2,open('./user_dic_uncertain.txt', 'r', encoding='utf-8') as file3:
    file1_line = file1.readlines()
    file2_line = file2.readlines()
    file3_line = file3.readlines()
    combined_content = file1.read() + file2.read()

# 합친 내용을 새로운 파일에 저장
with open('./combined_user_dic.txt', 'w', encoding='utf-8') as combined_file:
    combined_file.write(combined_content)

def nongdam(file_path):

    df, nltk_df = smi_to_df(file_path)
    # 특수기호 제거, 한글 숫자만
    #df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
    df['Text_kor'] = df['Text'].str.replace("[.,]"," ")

    # 문장 길이 기준으로 일정 길이 이상인 행만 선택
    min_sentence_length = 10
    df = df[df['Text'].apply(len) >= min_sentence_length]
    
    # 결측치 제거
    rows_to_remove = df[(df['Text_kor'] == ' ')].index
    df = df.drop(rows_to_remove)
    df.dropna(inplace=True)

    ## 문장 토큰화
    komoran = Komoran()
    # 사용자 용어사전 등록
    komoran = Komoran(userdic='./combined_user_dic.txt')

    tokenized_words, sentence_nouns_total = [], []

    for sentence in df['Text_kor']:
        # 토큰화 및 정규화
        tokenized_word = komoran.morphs(sentence)
        
        # 불용어 제거
        stopwords_removed_sentence = [word for word in tokenized_word if not word in stopwords]
        tokenized_words.append(stopwords_removed_sentence)

        # 명사만 추출
        nouns = komoran.nouns(sentence)
        
        # 불용어가 제거된 명사만 저장
        nouns_removed_stopwords = [noun for noun in nouns if not noun in stopwords]
        nouns_filtered = [noun for noun in nouns_removed_stopwords if len(noun) >= 2]
        sentence_nouns_total.append(nouns_filtered)
    
    df['tokenized_words'] = tokenized_words
    # 리스트로 되어 있는것을 단어로 구분
    df['tokenized_words'] = df['tokenized_words'].apply(lambda noun_list: ' '.join(map(str, noun_list)))

    list1 = []
    all_reviews = [item for sublist in sentence_nouns_total for item in sublist]

    word_freq = Counter(all_reviews)
    top_words = word_freq.most_common()

    for word, freq in top_words:
        if freq >= 10:
            list1.append(word)
    
    # 강의 단어 리스트 작성
    ## 강의 관련 없는 단어
    unrelated_list = [line.split('\t')[0] for line in file2_line]
    unrelated_list.append(file_path.split('/')[-3])
    ## 구분 애매한 단어
    ambiguous_list = [line.split('\t')[0] for line in file3_line]
    ## 강의 연관 단어
    list4 = [line.split('\t')[0] for line in file1_line]
    relation_total = list1 + list4
    relation_total = [word for word in relation_total if word not in (unrelated_list + ambiguous_list)]

    df['강의관련'] = df['tokenized_words'].apply(lambda text: sum(word.lower() in relation_total for word in text.split()))
    df['강의상관없는 단어'] = df['tokenized_words'].apply(lambda text: sum(word.lower() in unrelated_list for word in text.split()))
    df['애매한 단어'] = df['tokenized_words'].apply(lambda text: sum(word.lower() in ambiguous_list for word in text.split()))

    df['환산 점수'] = df['강의관련'] * 3 + df['강의상관없는 단어'] * -3 + df['애매한 단어']

    return df, len(df), len(df[df['환산 점수']<=2])
    
def visualization_df (file_path, t_name = ''):
    
    df, total_len, target_len =  nongdam(file_path)

    labels = ['강의 관련 문장', '관련없는 문장']
    values = [total_len - target_len, target_len]
    pull = [0, 0.1]

    trace = go.Pie(labels=labels, values=values, textinfo='label+percent', pull=pull)

    fig = go.Figure(data=trace)

    # 레이아웃 설정
    fig.update_layout(
        title=f'<{t_name}>강의와 상관없는 단어 비율',
        showlegend=True,
        height=500,  # 그림의 높이 설정
        width=700,   # 그림의 너비 설정
    )

    return fig.show()

def get_smi_file_names(path):
    
    
    smi_file_names = []
    for file in os.listdir(path):
        if file.endswith(".smi") and os.path.isfile(os.path.join(path, file)):
            smi_file_names.append(file)
    return smi_file_names

In [31]:
smi_file_path = "../강의스크립트/수학/서지나/[올림포스] 확률과 통계/"
smi_files = get_smi_file_names(smi_file_path)

list_append = []
for file_name in smi_files:
    df, total_len, target_len = nongdam(smi_file_path + file_name)
    T_name = smi_file_path.split('/')[-3]
    class_name = smi_file_path.split('/')[-2]
    list_append.append(['수학',T_name, class_name, file_name,total_len, target_len])

C:\Users\user\AppData\Local\Temp\ipykernel_8752\1847529638.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
C:\Users\user\AppData\Local\Temp\ipykernel_8752\1847529638.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
C:\Users\user\AppData\Local\Temp\ipykernel_8752\1847529638.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
C:\Users\user\AppData\Local\Temp\ipykernel_8752\1847529638.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´

In [32]:
df = pd.DataFrame(list_append, columns=['과목', '선생님이름','강좌명', 'file_name', 'total_len', 'taget_len'])
df

,과목,선생님이름,강좌명,file_name,total_len,taget_len
0,수학,서지나,[올림포스] 확률과 통계,10강. 이항정리(4)_(고2-기본).smi,572,210
1,수학,서지나,[올림포스] 확률과 통계,11강. 이항정리(5)_(고2-기본).smi,426,124
2,수학,서지나,[올림포스] 확률과 통계,12강. 이항정리(6)_(고2-기본).smi,541,183
3,수학,서지나,[올림포스] 확률과 통계,13강. 1단원 대단원 종합문제(1)_(고2-기본).smi,635,239
4,수학,서지나,[올림포스] 확률과 통계,14강. 1단원 대단원 종합문제(2)_(고2-기본).smi,444,116
5,수학,서지나,[올림포스] 확률과 통계,15강. 1단원 대단원 종합문제(3)_(고2-기본).smi,501,187
6,수학,서지나,[올림포스] 확률과 통계,16강. 확률의 뜻과 활용(1)_(고2-기본).smi,551,187
7,수학,서지나,[올림포스] 확률과 통계,17강. 확률의 뜻과 활용(2)_(고2-기본).smi,444,93
8,수학,서지나,[올림포스] 확률과 통계,18강. 확률의 뜻과 활용(3)_(고2-기본).smi,521,220
9,수학,서지나,[올림포스] 확률과 통계,19강. 확률의 뜻과 활용(4)_(고2-기본).smi,594,221


In [33]:
# 데이터프레임을 CSV 파일로 저장
csv_file_path = "서지나.csv"
df.to_csv(csv_file_path, index=False, encoding="utf-8")

# 전체 데이터 시각화

In [5]:
smi_file_path = "../유튜브 스크립트/[2021 수능특강] 정승제 확률과 통계/"
smi_files = get_smi_file_names(smi_file_path)

list_append = []
for file_name in smi_files:
    df, total_len, target_len = nongdam(smi_file_path + file_name)
    
    list_append.append([file_name,total_len, target_len])

data = pd.DataFrame(list_append, columns = ['파일명', '전체 문장 개수', '타겟 개수'])
data

t_name = '정승제'
total_len = data['전체 문장 개수'].sum()
target_len = data['타겟 개수'].sum()

labels = ['강의 관련 문장', '관련없는 문장']
values = [total_len - target_len, target_len]
pull = [0, 0.1]

trace = go.Pie(labels=labels, values=values, textinfo='label+percent', pull=pull)

fig = go.Figure(data=trace)

# 레이아웃 설정
fig.update_layout(
    title=f'<{t_name}>강의와 상관없는 단어 비율',
    showlegend=True,
    height=500,  # 그림의 높이 설정
    width=700,   # 그림의 너비 설정
)

fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_17088\2516668489.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
C:\Users\user\AppData\Local\Temp\ipykernel_17088\2516668489.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
C:\Users\user\AppData\Local\Temp\ipykernel_17088\2516668489.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text_kor'] = df['Text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9√+\-×⁰¹²³⁴⁵⁶⁷⁸⁹fxFXnN()´xyz/]"," ")
